In [1]:
import numpy as np
from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [12]:
data = pd.read_csv("train.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X= dataset
Y = dataset
X = X[:,1:2501]
Y = Y[:,0]

In [13]:
X_train = X[0:4800,:]
X_train = X_train/255.
X_test = X[4801:,:]
X_test = X_test/255.

In [14]:
Y = Y.reshape(Y.shape[0],1)
Y_train = Y[0:4800,:]
Y_train = Y_train.T
Y_test = Y[4801:,:]
Y_test = Y_test.T

In [15]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4800, 2500)
(1198, 2500)
(1, 4800)
(1, 1198)


In [16]:
image_x = 50
image_y = 50

train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test = X_test.reshape(X_test.shape[0],image_x,image_y,1)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4800, 50, 50, 1)
(1198, 50, 50, 1)
(1, 4800)
(1, 1198)


In [18]:
print(train_y.shape)

(4800, 6)


In [19]:
def keras_model(image_x,image_y):
    num_of_classes = 6
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size=(5,5), input_shape=(image_x,image_y,1),activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Conv2D(64,(5,5),activation ='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes,activation='softmax'))
    
    model.compile(loss ='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath= 'hand_gestures.h5'
    checkpoint1 = ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')
    callbacks_list = [checkpoint1]
    
    return model,callbacks_list

In [23]:
model, callbacks_list= keras_model(image_x,image_y)
model.fit(X_train,train_y, validation_data=(X_test,test_y),epochs=5,batch_size = 64,callbacks=callbacks_list)
scores = model.evaluate(X_test,test_y,verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
print_summary(model)
model.save('hand_gestures.h5')

Train on 4800 samples, validate on 1198 samples
Epoch 1/5
4800/4800 [==============================] - 148s 31ms/step - loss: 0.3666 - acc: 0.8765 - val_loss: 0.0766 - val_acc: 0.9791

Epoch 00001: val_acc improved from -inf to 0.97913, saving model to hand_gestures.h5
Epoch 2/5
4800/4800 [==============================] - 149s 31ms/step - loss: 0.0582 - acc: 0.9856 - val_loss: 0.0372 - val_acc: 0.9900

Epoch 00002: val_acc improved from 0.97913 to 0.98998, saving model to hand_gestures.h5
Epoch 3/5
4800/4800 [==============================] - 150s 31ms/step - loss: 0.0419 - acc: 0.9892 - val_loss: 0.0300 - val_acc: 0.9925

Epoch 00003: val_acc improved from 0.98998 to 0.99249, saving model to hand_gestures.h5
Epoch 4/5
4800/4800 [==============================] - 137s 28ms/step - loss: 0.0304 - acc: 0.9906 - val_loss: 0.0236 - val_acc: 0.9942

Epoch 00004: val_acc improved from 0.99249 to 0.99416, saving model to hand_gestures.h5
Epoch 5/5
4800/4800 [==============================] - 